In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, clear_output
try:
    import contourpy as cpy
except Exception:
    cpy = None
from functools import lru_cache

In [2]:
def f_paraboloid(x, y):
    return -0.12 * (x**2 + y**2) + 1

def f_sine_product_n1(x, y):
    return np.sin(1/2 * np.pi * x) * np.sin(1/2 * np.pi * y)

In [3]:
surface_funcs = {
    "Paraboloid": f_paraboloid,
    "Sine product": f_sine_product_n1,
}
x = np.linspace(-3.0, 3.0, 160)
y = np.linspace(-3.0, 3.0, 160)
X, Y = np.meshgrid(x, y)
_default_key = "Paraboloid"
# Cache surface data per function key: Z, grads, stats, contour generator
_surface_cache: dict[str, dict] = {}
# Initialize from default
Z = surface_funcs[_default_key](X, Y)
zmin, zmax = float(Z.min()), float(Z.max())
_cg_main = None
def _build_or_get_cache(key: str) -> dict:
    entry = _surface_cache.get(key)
    if entry is None:
        f = surface_funcs[key]
        Z_local = f(X, Y)
        zmin_local, zmax_local = float(Z_local.min()), float(Z_local.max())
        cg = None
        if cpy is not None:
            try:
                cg = cpy.contour_generator(x=x, y=y, z=Z_local, name="serial")
            except Exception:
                cg = None
        entry = {
            "Z": Z_local,
            "zmin": zmin_local,
            "zmax": zmax_local,
            "cg": cg,
            # grads are computed lazily
            "dZ_dx": None,
            "dZ_dy": None,
        }
        _surface_cache[key] = entry
    return entry
def compute_level_set_polylines(level: float) -> list[np.ndarray]:
    # Use contourpy when available for faster polyline extraction
    try:
        entry = _build_or_get_cache(surface_dropdown.value)
        cg = entry.get("cg")
        if cpy is not None and cg is not None:
            lines = cg.lines(float(level))
            return [np.asarray(seg, dtype=float) for seg in lines if np.asarray(seg).shape[0] > 1]
    except Exception:
        pass
    # Fallback: Matplotlib contour path extraction
    fig, ax = plt.subplots()
    cs = ax.contour(x, y, Z, levels=[level])
    paths: list[np.ndarray] = []
    try:
        if hasattr(cs, "allsegs") and cs.allsegs and len(cs.allsegs[0]) > 0:
            for seg in cs.allsegs[0]:
                v = np.asarray(seg)
                if v.shape[0] > 1:
                    paths.append(v)
        elif hasattr(cs, "collections") and cs.collections:
            for p in cs.collections[0].get_paths():
                v = p.vertices
                if v.shape[0] > 1:
                    paths.append(v)
    finally:
        plt.close(fig)
    return paths
def get_current_f():
    return surface_funcs[surface_dropdown.value]
def _get_surface_grads(entry: dict) -> tuple[np.ndarray, np.ndarray]:
    if entry["dZ_dx"] is None or entry["dZ_dy"] is None:
        dZ_dy, dZ_dx = np.gradient(entry["Z"], y, x)
        entry["dZ_dx"], entry["dZ_dy"] = dZ_dx, dZ_dy
    return entry["dZ_dx"], entry["dZ_dy"]
def add_gradient_field_flat(fig: go.Figure, density: int = 12, arrow_color: str = "#1f77b4", arrow_length: float = 0.6, head_length_frac: float = 0.25, head_angle_deg: float = 28.0, line_width: int = 6) -> None:
    # Ensure Z reflects the current surface and get cached grads
    _update_z_stats_for_current_surface()
    entry = _build_or_get_cache(surface_dropdown.value)
    dZ_dx, dZ_dy = _get_surface_grads(entry)
    ny, nx = entry["Z"].shape
    step_x = max(1, nx // density)
    step_y = max(1, ny // density)
    xs = X[::step_y, ::step_x]
    ys = Y[::step_y, ::step_x]
    fx_sampled = dZ_dx[::step_y, ::step_x]
    fy_sampled = dZ_dy[::step_y, ::step_x]
    mags = np.sqrt(fx_sampled * fx_sampled + fy_sampled * fy_sampled) + 1e-9
    ux = fx_sampled / mags
    uy = fy_sampled / mags
    # Prepare multi-segment lines with NaN breaks
    z_floor = float(zmin + 1e-6)
    x_lines = []
    y_lines = []
    z_lines = []
    x_heads = []
    y_heads = []
    z_heads = []
    head_len = float(arrow_length * head_length_frac)
    theta = float(np.deg2rad(head_angle_deg))
    cos_t, sin_t = float(np.cos(theta)), float(np.sin(theta))
    def rot(u, v, c, s):
        return u * c - v * s, u * s + v * c
    for j in range(xs.shape[0]):
        for i in range(xs.shape[1]):
            x0 = float(xs[j, i])
            y0 = float(ys[j, i])
            dx = float(ux[j, i])
            dy = float(uy[j, i])
            x1 = x0 + arrow_length * dx
            y1 = y0 + arrow_length * dy
            x_lines.extend([x0, x1, np.nan])
            y_lines.extend([y0, y1, np.nan])
            z_lines.extend([z_floor, z_floor, np.nan])
            rx1, ry1 = rot(dx, dy, cos_t, sin_t)
            rx2, ry2 = rot(dx, dy, cos_t, -sin_t)
            x_heads.extend([x1, x1 - head_len * rx1, np.nan])
            y_heads.extend([y1, y1 - head_len * ry1, np.nan])
            z_heads.extend([z_floor, z_floor, np.nan])
            x_heads.extend([x1, x1 - head_len * rx2, np.nan])
            y_heads.extend([y1, y1 - head_len * ry2, np.nan])
            z_heads.extend([z_floor, z_floor, np.nan])
    fig.add_trace(go.Scatter3d(x=x_lines,y=y_lines,z=z_lines,mode="lines",line=dict(color=arrow_color, width=line_width),name="Gradient field",showlegend=True,))
    fig.add_trace(go.Scatter3d(x=x_heads,y=y_heads,z=z_heads,mode="lines",line=dict(color=arrow_color, width=line_width),name="",showlegend=False,))
def build_3d_figure(level_z: float, show_plane: bool, plane_z: float) -> go.Figure:
    fig = go.Figure()
    # Main surface
    fig.add_trace(go.Surface(x=X,y=Y,z=Z,colorscale="Viridis",reversescale=False,showscale=False,colorbar=dict(title="Height"),name="Surface",opacity=0.55,))
    # Optional horizontal plane at z = plane_z
    if show_plane:
        plane_z_arr = np.full_like(Z, plane_z)
        fig.add_trace(go.Surface(x=X,y=Y,z=plane_z_arr,colorscale=[[0, "#AAAAAA"], [1, "#AAAAAA"]],showscale=False,opacity=0.30,name=f"Plane z={plane_z:.2f}",))
    # Highlight the intersection contour at the selected level
    level_paths = compute_level_set_polylines(level_z)
    # Add red level set on the 3D surface
    if show_bottom_redlevel_chk.value:
        f = get_current_f()
        for verts in level_paths:
            # Compute z values on the surface for each vertex
            z_surface = np.array([float(f(v[0], v[1])) for v in verts])
            fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=z_surface,mode="lines",line=dict(color="#FF4136", width=3),name="Selected level (surface)",showlegend=False,))
    # Bottom content selection driven by new checkboxes
    z_floor = zmin
    if show_bottom_heatmap_chk.value:
        fig.add_trace(go.Surface(x=X,y=Y,z=np.full_like(Z, z_floor),surfacecolor=Z,cmin=zmin,cmax=zmax,colorscale="Viridis",showscale=False,opacity=0.4,name="Topo floor",hoverinfo="skip",))
        if zmax == zmin:
            selected_levels = [zmin]
        else:
            z_span = (zmax - zmin)
            selected_levels = list(zmin + np.linspace(0.1, 0.9, 6) * z_span)
        for lvl in selected_levels:
            for verts in compute_level_set_polylines(lvl):
                fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3), mode="lines",line=dict(color="#555555", width=5),name="Topo contours",showlegend=False,))
    if show_bottom_arrows_chk.value:
        add_gradient_field_flat(fig, density=12, arrow_color="#1f77b4", arrow_length=0.2, head_length_frac=0.28, head_angle_deg=26.0, line_width=6)
    if show_bottom_redlevel_chk.value:
        for verts in level_paths:
            fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3),mode="lines",line=dict(color="#FF4136", width=2),name="Selected level (floor)",showlegend=False,))
    eps_z = float(max(1e-6, 1e-3 * (zmax - zmin)))
    scene = dict(xaxis_title="x",yaxis_title="y",zaxis_title="z",xaxis=dict(showspikes=False),yaxis=dict(showspikes=False),zaxis=dict(showspikes=False, range=[zmin, zmax + eps_z]),aspectmode="data",
    )
    # Always use normal 3D perspective view (2D top-down view is shown separately)
    fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=1.35, y=1.35, z=0.95), projection=dict(type="orthographic"))), margin=dict(l=0, r=0, t=100, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Visual Representation of the Gradient",width=780,height=780,uirevision="main-3d")
    return fig
instructions = widgets.HTML(
    value=(
        "<b>How to use:</b>"
        "<ul>"
        "<li>Select a surface from the dropdown to switch functions.</li>"
        "<li>Rotate/zoom the 3D surface. It's colored by height, with the intersection contour highlighted in red.</li>"
        "<li>Optionally show a floor topo heatmap + contours to view from above.</li>"
        "<li>Enter a point (x0, y0) and press Enter to add tangent lines, the tangent plane, and the normal line at that point.</li>"
        "</ul>"
    )
)
surface_dropdown = widgets.Dropdown(options=list(surface_funcs.keys()),value=_default_key,description="Surface",layout=widgets.Layout(width="280px"),
)
z_slider = widgets.FloatSlider(
    description="Level/Plane z",min=zmin,max=zmax,step=(zmax - zmin) / 200.0 if zmax > zmin else 0.01,value=(zmin + zmax) / 2.0,continuous_update=False,readout_format=".2f",layout=widgets.Layout(width="350px"),
)
show_plane_chk = widgets.Checkbox(value=False, description="Show plane")
birds_eye_toggle = widgets.ToggleButton(
    value=False, description="Bird’s-eye 2D view", icon="eye"
)
bottom_mode_dd = widgets.Dropdown(options=["No Bottom Floor", "Level set heatmap", "Gradient vector field", "Heatmap + gradient field"],value="No Bottom Floor",description="Bottom",layout=widgets.Layout(width="350px"),
)
# Hide legacy dropdown in favor of mix-and-match controls
bottom_mode_dd.layout.display = "none"
# New: mix-and-match bottom plane controls ("Alter Bottom Plane")
show_bottom_heatmap_chk = widgets.Checkbox(value=False, description="Heatmap")
show_bottom_arrows_chk = widgets.Checkbox(value=True, description="Gradient field")
show_bottom_redlevel_chk = widgets.Checkbox(value=True, description="Selected level (red)")
bottom_table_title = widgets.HTML("<b>Alter Bottom Plane</b>")
bottom_table = widgets.VBox([widgets.HBox([show_bottom_heatmap_chk, widgets.HTML("Level sets heatmap")], layout=widgets.Layout(align_items="center")),widgets.HBox([show_bottom_arrows_chk, widgets.HTML("Gradient vector field")], layout=widgets.Layout(align_items="center")),widgets.HBox([show_bottom_redlevel_chk, widgets.HTML("Red level set projection")], layout=widgets.Layout(align_items="center")),
], layout=widgets.Layout(align_items="flex-start"))
# New: lock the level to f(x0,y0)
lock_level_chk = widgets.Checkbox(value=True, description="Lock level set to f(x0,y0)")
x0_input = widgets.FloatSlider(description="x0", min=float(x.min()), max=float(x.max()), step=0.02, value=2.3, readout_format=".2f", continuous_update=False, layout=widgets.Layout(width="300px"))
y0_input = widgets.FloatSlider(description="y0", min=float(y.min()), max=float(y.max()), step=0.02, value=0.6, readout_format=".2f", continuous_update=False, layout=widgets.Layout(width="300px"))
# New: toggle for cone arrowheads (off by default for performance)
show_cones_chk = widgets.Checkbox(value=False, description="Show arrowheads (cones)")
out3d = widgets.Output()
out3d.layout = widgets.Layout(width="780px", height="780px")
out2d = widgets.Output()
out2d.layout = widgets.Layout(width="260px", height="260px")
current_fig3d = None
current_fig2d = None
def _update_z_stats_for_current_surface():
    global Z, zmin, zmax, _cg_main
    # Get or build cache for current surface
    entry = _build_or_get_cache(surface_dropdown.value)
    Z = entry["Z"]
    zmin, zmax = entry["zmin"], entry["zmax"]
    _cg_main = entry.get("cg")
    # update unified slider
    z_slider.min = zmin
    z_slider.max = zmax
    z_slider.step = (zmax - zmin) / 200.0 if zmax > zmin else 0.01
    if z_slider.value < zmin or z_slider.value > zmax:
        z_slider.value = (zmin + zmax) / 2.0
is_rendering_main = False
def render_all():
    global current_fig3d, current_fig2d, is_rendering_main
    if is_rendering_main:
        return
    is_rendering_main = True
    _update_z_stats_for_current_surface()
    # Determine the level to use
    try:
        x0v = float(x0_input.value)
        y0v = float(y0_input.value)
    except Exception:
        x0v, y0v = 0.0, 0.0
    if lock_level_chk.value:
        try:
            f = get_current_f()
            level_val = float(np.clip(f(x0v, y0v), zmin, zmax))
        except Exception:
            level_val = float(np.clip((zmin + zmax) / 2.0, zmin, zmax))
        z_slider.layout.display = "none"
    else:
        level_val = z_slider.value
        z_slider.layout.display = "flex"
    current_fig3d = build_3d_figure(
        level_z=level_val,
        show_plane=show_plane_chk.value,
        plane_z=level_val,
    )
    try:
        if np.isfinite(x0v) and np.isfinite(y0v):
            f = get_current_f()
            z0 = float(f(x0v, y0v))
            z_plot = float(z0 + max(1e-6, 1e-3 * (zmax - zmin)))
            z_floor = zmin + 1e-3
            # Point on surface
            current_fig3d.add_trace(
                go.Scatter3d(
                    x=[x0v], y=[y0v], z=[z_plot],
                    mode="markers",
                    marker=dict(size=6, color="#111111"),
                    name="Point (x0, y0, f)"
                )
            )
            # Projection marker on bottom plane
            current_fig3d.add_trace(
                go.Scatter3d(
                    x=[x0v], y=[y0v], z=[z_floor],
                    mode="markers",
                    marker=dict(size=6, color="#111111"),
                    name="Point projection"
                )
            )
            # Dashed line connecting point to projection
            current_fig3d.add_trace(
                go.Scatter3d(
                    x=[x0v, x0v], y=[y0v, y0v], z=[z_plot, z_floor],
                    mode="lines",
                    line=dict(color="rgba(0,0,0,0.3)", width=2, dash="dash"),
                    name="",
                    showlegend=False
                )
            )
    except Exception:
        pass
    with out3d:
        clear_output(wait=True)
        display(current_fig3d)
    # Also update the 2D view to show the current surface with current point from sliders
    # Only show path if it exists, but always show current point from sliders
    try:
        if np.isfinite(x0v) and np.isfinite(y0v):
            current_fig2d = _build_2d_path_figure(show_current_point=(x0v, y0v))
        else:
            current_fig2d = _build_2d_path_figure()
    except Exception:
        current_fig2d = _build_2d_path_figure()
    with out2d:
        clear_output(wait=True)
        display(current_fig2d)
    is_rendering_main = False

def _on_surface_change(change):
    """Clear path when surface changes and re-render"""
    global path_x, path_y, path_z, selected_level_val
    path_x.clear()
    path_y.clear()
    path_z.clear()
    selected_level_val = None
    render_all()

surface_dropdown.observe(_on_surface_change, names="value")
z_slider.observe(lambda change: render_all(), names="value")
show_plane_chk.observe(lambda change: render_all(), names="value")
# Observe new bottom toggles
show_bottom_heatmap_chk.observe(lambda change: render_all(), names="value")
show_bottom_arrows_chk.observe(lambda change: render_all(), names="value")
show_bottom_redlevel_chk.observe(lambda change: render_all(), names="value")
x0_input.observe(lambda change: render_all(), names="value")
y0_input.observe(lambda change: render_all(), names="value")
lock_level_chk.observe(lambda change: render_all(), names="value")
show_cones_chk.observe(lambda change: render_all(), names="value")
controls_row1 = widgets.HBox([
    surface_dropdown,
])
# Replace dropdown with the new "Alter Bottom Plane" table in the UI
plane_controls_mx = widgets.VBox([
    bottom_table_title,
    bottom_table,
])
point_row = widgets.HBox([widgets.HTML("<b>Point:</b>&nbsp;"),x0_input,y0_input])
run_btn = widgets.Button(description="Run Gradient Ascent", button_style="primary")
status_html = widgets.HTML(value="")

# Path state
path_x: list[float] = []
path_y: list[float] = []
path_z: list[float] = []
# Pinned selected level for bottom-plane red contour during ascent
selected_level_val: float | None = None
# Gradient vector scaling parameter: max length = 1/a
gradient_scale_a = 0.5  # Adjust this to change max gradient arrow length (1/a = 2.0)


def _current_grad(xv: float, yv: float) -> tuple[float, float]:
    key = surface_dropdown.value
    if key == "Paraboloid":
        # f = -0.12*(x^2+y^2) + 3  => grad f = (-0.24 x, -0.24 y)
        return (-0.24 * float(xv), -0.24 * float(yv))
    else:
        # f = sin((pi/2) x) sin((pi/2) y)
        k = np.pi / 2.0
        gx = float(np.cos(k * xv) * k * np.sin(k * yv))
        gy = float(np.sin(k * xv) * k * np.cos(k * yv))
        return (gx, gy)


def _add_gradient_vectors(fig: go.Figure, x0: float, y0: float, z0: float, z_floor: float, a: float = 0.5) -> None:
    """Add gradient vectors on surface and bottom plane with scaling ||grad||/(1 + a*||grad||)"""
    gx, gy = _current_grad(x0, y0)
    grad_mag = float(np.hypot(gx, gy))
    
    if grad_mag < 1e-12:
        return
    
    # Scaled length: ||grad||/(1 + a*||grad||)
    scaled_length = grad_mag / (1.0 + a * grad_mag)
    
    # Normalized direction
    dir_x = gx / grad_mag
    dir_y = gy / grad_mag
    
    # Gradient vector on bottom plane (2D)
    p0_floor = np.array([x0, y0, z_floor], dtype=float)
    p1_floor = np.array([x0 + scaled_length * dir_x, y0 + scaled_length * dir_y, z_floor], dtype=float)
    fig.add_trace(go.Scatter3d(
        x=[p0_floor[0], p1_floor[0]], 
        y=[p0_floor[1], p1_floor[1]], 
        z=[p0_floor[2], p1_floor[2]],
        mode="lines",
        line=dict(color="#AA00FF", width=8),
        name="Gradient (floor)",
        showlegend=False
    ))
    # Arrowhead on bottom plane (cone pointing in gradient direction, flat on z)
    try:
        cone_size = min(0.15, scaled_length * 0.2)  # Scale cone size with arrow length
        fig.add_trace(go.Cone(
            x=[p1_floor[0]], y=[p1_floor[1]], z=[p1_floor[2]],
            u=[dir_x * cone_size], v=[dir_y * cone_size], w=[0.0],
            anchor="tip",
            colorscale=[[0, "#AA00FF"], [1, "#AA00FF"]],
            showscale=False,
            sizemode="absolute",
            sizeref=cone_size,
            name="",
        ))
    except Exception:
        pass
    
    # Gradient vector on surface (tangent to surface)
    # Move in gradient direction in x-y plane, compute actual z from surface
    p0_surf = np.array([x0, y0, z0], dtype=float)
    # End point in x-y plane
    x1 = x0 + scaled_length * dir_x
    y1 = y0 + scaled_length * dir_y
    # Compute actual z value on surface at new point
    f = surface_funcs[surface_dropdown.value]
    z1 = float(f(x1, y1))
    p1_surf = np.array([x1, y1, z1], dtype=float)
    # Direction vector for surface arrow (3D)
    dir_3d = p1_surf - p0_surf
    dir_3d_mag = float(np.linalg.norm(dir_3d))
    if dir_3d_mag > 1e-12:
        dir_3d_normalized = dir_3d / dir_3d_mag
    else:
        dir_3d_normalized = np.array([dir_x, dir_y, 0.0], dtype=float)
    fig.add_trace(go.Scatter3d(
        x=[p0_surf[0], p1_surf[0]], 
        y=[p0_surf[1], p1_surf[1]], 
        z=[p0_surf[2], p1_surf[2]],
        mode="lines",
        line=dict(color="#AA00FF", width=8),
        name="Gradient (surface)",
        showlegend=False
    ))
    # Arrowhead on surface (cone pointing in gradient direction, tangent to surface)
    try:
        cone_size = min(0.15, scaled_length * 0.2)  # Scale cone size with arrow length
        fig.add_trace(go.Cone(
            x=[p1_surf[0]], y=[p1_surf[1]], z=[p1_surf[2]],
            u=[dir_3d_normalized[0] * cone_size], 
            v=[dir_3d_normalized[1] * cone_size], 
            w=[dir_3d_normalized[2] * cone_size],
            anchor="tip",
            colorscale=[[0, "#AA00FF"], [1, "#AA00FF"]],
            showscale=False,
            sizemode="absolute",
            sizeref=cone_size,
            name="",
        ))
    except Exception:
        pass


def _build_2d_path_figure(path_x_snap=None, path_y_snap=None, show_current_point=None) -> go.Figure:
    # Background heatmap + contours
    fig2 = go.Figure()
    # Recompute Z for current surface
    f = surface_funcs[surface_dropdown.value]
    Z_local = f(X, Y)
    fig2.add_trace(go.Heatmap(z=Z_local, x=x, y=y, colorscale="Viridis", showscale=False))
    fig2.add_trace(go.Contour(z=Z_local, x=x, y=y, showscale=False,
                              contours=dict(coloring="none", showlines=True),
                              line=dict(color="#777777", width=1)))
    # Use provided snapshot or fall back to global path
    px = path_x_snap if path_x_snap is not None else path_x
    py = path_y_snap if path_y_snap is not None else path_y
    # Path tail - only show if path exists
    if len(px) >= 2:
        fig2.add_trace(go.Scatter(x=px, y=py, mode="lines",
                                  line=dict(color="#e31a1c", width=3), name="path"))
    # Current point - show from path if available, otherwise show from sliders
    if show_current_point is not None:
        # Explicitly provided current point (from sliders)
        x_curr, y_curr = show_current_point
        fig2.add_trace(go.Scatter(x=[x_curr], y=[y_curr], mode="markers",
                                  marker=dict(size=8, color="#111111"), name="x(t)"))
    elif len(px) >= 1:
        # Use last point from path
        fig2.add_trace(go.Scatter(x=[px[-1]], y=[py[-1]], mode="markers",
                                  marker=dict(size=8, color="#111111"), name="x(t)"))
    fig2.update_layout(xaxis_title="x", yaxis_title="y", title="Ascent path (2D)",
                       width=260, height=260, margin=dict(l=40, r=10, t=40, b=40), showlegend=False)
    return fig2


def _render_with_path():
    global selected_level_val
    # Update selected level to the current point's z value so red level set moves with the point
    if len(path_z) >= 1:
        selected_level_val = float(path_z[-1])
    # Rebuild 3D base with pinned selected level if available
    lvl = selected_level_val if selected_level_val is not None else (z_slider.value)
    fig = build_3d_figure(level_z=lvl,
                          show_plane=show_plane_chk.value,
                          plane_z=lvl)
    z_floor = zmin + 1e-3
    # Tail on surface
    if len(path_x) >= 2:
        fig.add_trace(go.Scatter3d(x=path_x, y=path_y, z=path_z, mode="lines",
                                   line=dict(color="#e31a1c", width=3), name="ascent path"))
        # Projection of path tail to bottom plane
        path_z_floor = [z_floor] * len(path_x)
        fig.add_trace(go.Scatter3d(x=path_x, y=path_y, z=path_z_floor, mode="lines",
                                   line=dict(color="#e31a1c", width=2), name="ascent path (projection)"))
    # Current point (lifted slightly above surface to remain visible at max)
    if len(path_x) >= 1:
        eps_z = float(max(1e-6, 1e-3 * (zmax - zmin)))
        z_pt = float(path_z[-1] + eps_z)
        # Point on surface
        fig.add_trace(go.Scatter3d(x=[path_x[-1]], y=[path_y[-1]], z=[z_pt], mode="markers",
                                   marker=dict(size=6, color="#111111"), name="x(t)"))
        # Projection marker on bottom plane
        fig.add_trace(go.Scatter3d(x=[path_x[-1]], y=[path_y[-1]], z=[z_floor], mode="markers",
                                   marker=dict(size=6, color="#111111"), name="x(t) projection"))
        # Dashed line connecting point to projection
        fig.add_trace(go.Scatter3d(x=[path_x[-1], path_x[-1]], y=[path_y[-1], path_y[-1]], 
                                   z=[z_pt, z_floor], mode="lines",
                                   line=dict(color="rgba(0,0,0,0.3)", width=2, dash="dash"),
                                   name="", showlegend=False))
        # Add gradient vectors on surface and bottom plane
        _add_gradient_vectors(fig, path_x[-1], path_y[-1], path_z[-1], z_floor, gradient_scale_a)
    with out3d:
        clear_output(wait=True)
        display(fig)
    with out2d:
        clear_output(wait=True)
        display(_build_2d_path_figure())


def _run_ascent_clicked(_):
    # Initialize path from current slider values
    global selected_level_val
    try:
        x_curr = float(x0_input.value)
        y_curr = float(y0_input.value)
    except Exception:
        x_curr, y_curr = 0.0, 0.0
    f = surface_funcs[surface_dropdown.value]
    z_curr = float(f(x_curr, y_curr))
    # Pin the selected level to the initial level for stable bottom-plane red contour
    selected_level_val = float(z_curr)
    path_x.clear(); path_y.clear(); path_z.clear()
    path_x.append(x_curr); path_y.append(y_curr); path_z.append(z_curr)
    status_html.value = "Starting gradient ascent..."
    # Euler stepping params
    dt = 0.1
    num_steps = 100
    # Domain bounds
    xmin, xmax = float(x.min()), float(x.max())
    ymin, ymax = float(y.min()), float(y.max())
    import time
    for k in range(num_steps):
        gx, gy = _current_grad(x_curr, y_curr)
        x_curr = float(np.clip(x_curr + dt * gx, xmin, xmax))
        y_curr = float(np.clip(y_curr + dt * gy, ymin, ymax))
        z_curr = float(f(x_curr, y_curr))
        path_x.append(x_curr); path_y.append(y_curr); path_z.append(z_curr)
        status_html.value = f"Step {k+1} / {num_steps} (x={x_curr:.2f}, y={y_curr:.2f}, z={z_curr:.2f})"
        # Update view intermittently for smoother animation
        if (k % 3) == 0 or k == num_steps - 1:
            _render_with_path()
            time.sleep(0.01)
    status_html.value = f"Gradient ascent complete after {num_steps} steps."


run_btn.on_click(_run_ascent_clicked)

ui = widgets.VBox([controls_row1,plane_controls_mx,point_row,widgets.HBox([run_btn, status_html]),widgets.HBox([out3d,out2d]),
])

In [4]:
#Gradient Ascent: Animation Version
render_all()
display(ui)

In [ ]:
#Gradient Ascent: Slider version
# Same widgets and visualizations as animation version, but with a slider instead of button

# Create separate widgets for slider version
surface_dropdown_slider = widgets.Dropdown(options=list(surface_funcs.keys()),value=_default_key,description="Surface",layout=widgets.Layout(width="280px"))
x0_input_slider = widgets.FloatSlider(description="x0", min=float(x.min()), max=float(x.max()), step=0.02, value=2.3, readout_format=".2f", continuous_update=False, layout=widgets.Layout(width="300px"))
y0_input_slider = widgets.FloatSlider(description="y0", min=float(y.min()), max=float(y.max()), step=0.02, value=0.6, readout_format=".2f", continuous_update=False, layout=widgets.Layout(width="300px"))
show_bottom_heatmap_chk_slider = widgets.Checkbox(value=False, description="Heatmap")
show_bottom_arrows_chk_slider = widgets.Checkbox(value=True, description="Gradient field")
show_bottom_redlevel_chk_slider = widgets.Checkbox(value=True, description="Selected level (red)")
step_slider = widgets.IntSlider(description="Step", min=0, max=100, value=0, continuous_update=False, layout=widgets.Layout(width="400px"))
status_html_slider = widgets.HTML(value="")

# Output widgets
out3d_slider = widgets.Output()
out3d_slider.layout = widgets.Layout(width="780px", height="780px")
out2d_slider = widgets.Output()
out2d_slider.layout = widgets.Layout(width="260px", height="260px")

# Path storage for slider version
path_x_slider: list[float] = []
path_y_slider: list[float] = []
path_z_slider: list[float] = []
selected_level_val_slider: float | None = None

def _current_grad_slider(xv: float, yv: float) -> tuple[float, float]:
    """Compute gradient for slider version using its own surface dropdown"""
    key = surface_dropdown_slider.value
    if key == "Paraboloid":
        # f = -0.12*(x^2+y^2) + 3  => grad f = (-0.24 x, -0.24 y)
        return (-0.24 * float(xv), -0.24 * float(yv))
    else:
        # f = sin((pi/2) x) sin((pi/2) y)
        k = np.pi / 2.0
        gx = float(np.cos(k * xv) * k * np.sin(k * yv))
        gy = float(np.sin(k * xv) * k * np.cos(k * yv))
        return (gx, gy)

def _compute_full_path_slider():
    """Compute the full gradient ascent path"""
    global path_x_slider, path_y_slider, path_z_slider, selected_level_val_slider
    try:
        x_curr = float(x0_input_slider.value)
        y_curr = float(y0_input_slider.value)
    except Exception:
        x_curr, y_curr = 0.0, 0.0
    
    # Use the slider version's surface dropdown
    f = surface_funcs[surface_dropdown_slider.value]
    z_curr = float(f(x_curr, y_curr))
    selected_level_val_slider = float(z_curr)
    
    path_x_slider.clear()
    path_y_slider.clear()
    path_z_slider.clear()
    path_x_slider.append(x_curr)
    path_y_slider.append(y_curr)
    path_z_slider.append(z_curr)
    
    # Euler stepping params with adaptive learning rate for slider version
    # Start with smaller learning rate, gradually increase to move faster at the end
    # Use higher learning rates for paraboloid, lower for sine product
    if surface_dropdown_slider.value == "Paraboloid":
        dt_min = 0.04  # Higher initial learning rate for paraboloid
        dt_max = 0.15  # Higher maximum learning rate for paraboloid
    else:
        dt_min = 0.01  # Small initial learning rate for slower start (sine product) - 2x slower
        dt_max = 0.05  # Maximum learning rate for faster end (sine product) - 2x slower
    num_steps = 100
    xmin, xmax = float(x.min()), float(x.max())
    ymin, ymax = float(y.min()), float(y.max())
    
    for k in range(num_steps):
        # Linear schedule: start small, increase over time
        # Progress from 0 to 1 over num_steps
        progress = float(k) / float(num_steps - 1) if num_steps > 1 else 0.0
        dt = dt_min + (dt_max - dt_min) * progress
        
        gx, gy = _current_grad_slider(x_curr, y_curr)
        x_curr = float(np.clip(x_curr + dt * gx, xmin, xmax))
        y_curr = float(np.clip(y_curr + dt * gy, ymin, ymax))
        z_curr = float(f(x_curr, y_curr))
        path_x_slider.append(x_curr)
        path_y_slider.append(y_curr)
        path_z_slider.append(z_curr)
    
    # Update slider max to match path length
    step_slider.max = len(path_x_slider) - 1

def _render_with_path_slider():
    """Render visualization showing path up to current slider step"""
    global Z, zmin, zmax, selected_level_val_slider
    # Update surface stats
    entry = _build_or_get_cache(surface_dropdown_slider.value)
    Z = entry["Z"]
    zmin, zmax = entry["zmin"], entry["zmax"]
    
    # Get current step from slider
    current_step = step_slider.value
    
    # Initialize path variables to avoid scope issues
    path_x_visible = []
    path_y_visible = []
    path_z_visible = []
    
    # Update selected level to the current step's z value so red level set moves with the point
    if current_step < len(path_z_slider):
        selected_level_val_slider = float(path_z_slider[current_step])
        # Path up to current step
        path_x_visible = path_x_slider[:current_step+1]
        path_y_visible = path_y_slider[:current_step+1]
        path_z_visible = path_z_slider[:current_step+1]
    
    # Determine level
    lvl = selected_level_val_slider if selected_level_val_slider is not None else (zmin + zmax) / 2.0
    
    # Build base figure (reuse build_3d_figure but need to adapt it)
    fig = go.Figure()
    # Main surface - ensure we use the correct Z from cache
    fig.add_trace(go.Surface(x=X,y=Y,z=Z,colorscale="Viridis",reversescale=False,showscale=False,colorbar=dict(title="Height"),name="Surface",opacity=0.55))
    
    # Calculate floor position - ensure it's below the surface but visible
    z_floor = zmin - 0.1 * (zmax - zmin) if zmax > zmin else zmin - 0.1
    # Bottom plane options
    if show_bottom_heatmap_chk_slider.value:
        fig.add_trace(go.Surface(x=X,y=Y,z=np.full_like(Z, z_floor),surfacecolor=Z,cmin=zmin,cmax=zmax,colorscale="Viridis",showscale=False,opacity=0.4,name="Topo floor",hoverinfo="skip"))
        if zmax == zmin:
            selected_levels = [zmin]
        else:
            z_span = (zmax - zmin)
            selected_levels = list(zmin + np.linspace(0.1, 0.9, 6) * z_span)
        # Temporarily set surface_dropdown to match slider version for level sets
        old_val = surface_dropdown.value
        surface_dropdown.value = surface_dropdown_slider.value
        _update_z_stats_for_current_surface()
        for lvl_contour in selected_levels:
            level_paths = compute_level_set_polylines(lvl_contour)
            for verts in level_paths:
                fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3), mode="lines",line=dict(color="#555555", width=5),name="Topo contours",showlegend=False))
        surface_dropdown.value = old_val
    
    if show_bottom_arrows_chk_slider.value:
        # Temporarily set surface_dropdown to match slider version for gradient field
        old_val = surface_dropdown.value
        surface_dropdown.value = surface_dropdown_slider.value
        _update_z_stats_for_current_surface()
        add_gradient_field_flat(fig, density=12, arrow_color="#1f77b4", arrow_length=0.2, head_length_frac=0.28, head_angle_deg=26.0, line_width=6)
        surface_dropdown.value = old_val
    
    if show_bottom_redlevel_chk_slider.value and selected_level_val_slider is not None:
        # Temporarily set surface_dropdown to match slider version for level sets
        old_val = surface_dropdown.value
        surface_dropdown.value = surface_dropdown_slider.value
        _update_z_stats_for_current_surface()
        level_paths = compute_level_set_polylines(selected_level_val_slider)
        surface_dropdown.value = old_val
        # Add red level set on the 3D surface
        f = surface_funcs[surface_dropdown_slider.value]
        for verts in level_paths:
            # Compute z values on the surface for each vertex
            z_surface = np.array([float(f(v[0], v[1])) for v in verts])
            fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=z_surface,mode="lines",line=dict(color="#FF4136", width=3),name="Selected level (surface)",showlegend=False))
        # Add red level set on the bottom plane
        for verts in level_paths:
            fig.add_trace(go.Scatter3d(x=verts[:, 0],y=verts[:, 1],z=np.full(verts.shape[0], z_floor + 1e-3),mode="lines",line=dict(color="#FF4136", width=2),name="Selected level (floor)",showlegend=False))
    
    if current_step < len(path_x_slider) and len(path_x_visible) > 0:
        # Path on surface
        if len(path_x_visible) >= 2:
            fig.add_trace(go.Scatter3d(x=path_x_visible, y=path_y_visible, z=path_z_visible, mode="lines",
                                       line=dict(color="#e31a1c", width=3), name="ascent path"))
            # Path projection on bottom plane
            path_z_floor_visible = [z_floor] * len(path_x_visible)
            fig.add_trace(go.Scatter3d(x=path_x_visible, y=path_y_visible, z=path_z_floor_visible, mode="lines",
                                       line=dict(color="#e31a1c", width=2), name="ascent path (projection)"))
        
        # Current point
        if len(path_x_visible) >= 1:
            eps_z = float(max(1e-6, 0.01 * (zmax - zmin)) if zmax > zmin else 0.01)
            z_pt = float(path_z_visible[-1] + eps_z)
            # Point on surface
            fig.add_trace(go.Scatter3d(x=[path_x_visible[-1]], y=[path_y_visible[-1]], z=[z_pt], mode="markers",
                                       marker=dict(size=6, color="#111111"), name="x(t)"))
            # Projection marker
            fig.add_trace(go.Scatter3d(x=[path_x_visible[-1]], y=[path_y_visible[-1]], z=[z_floor], mode="markers",
                                       marker=dict(size=6, color="#111111"), name="x(t) projection"))
            # Dashed line connecting point to projection
            fig.add_trace(go.Scatter3d(x=[path_x_visible[-1], path_x_visible[-1]], y=[path_y_visible[-1], path_y_visible[-1]], 
                                       z=[z_pt, z_floor], mode="lines",
                                       line=dict(color="rgba(0,0,0,0.3)", width=2, dash="dash"),
                                       name="", showlegend=False))
            # Gradient vectors (temporarily set surface_dropdown to match slider version)
            old_surface = surface_dropdown.value
            surface_dropdown.value = surface_dropdown_slider.value
            _add_gradient_vectors(fig, path_x_visible[-1], path_y_visible[-1], path_z_visible[-1], z_floor, gradient_scale_a)
            surface_dropdown.value = old_surface
    
    # Layout - ensure z-axis range includes floor, surface, and all visible data with proper padding
    # Collect all z-values that need to be visible
    z_values_to_include = [zmin, zmax, z_floor]
    
    # Add path z-values if path exists
    if len(path_z_visible) > 0:
        z_values_to_include.extend(path_z_visible)
        # Also include the lifted point if it exists
        if len(path_z_visible) >= 1:
            eps_z = float(max(1e-6, 0.01 * (zmax - zmin)) if zmax > zmin else 0.01)
            z_values_to_include.append(float(path_z_visible[-1] + eps_z))
    
    # Calculate adaptive range based on all visible data
    z_data_min = float(min(z_values_to_include))
    z_data_max = float(max(z_values_to_include))
    z_span = z_data_max - z_data_min
    
    # Add padding (5% on each side, with minimum padding)
    padding = max(0.05 * z_span, 0.1) if z_span > 0 else 0.1
    z_range_min = z_data_min - padding
    z_range_max = z_data_max + padding
    scene = dict(xaxis_title="x",yaxis_title="y",zaxis_title="z",xaxis=dict(showspikes=False),yaxis=dict(showspikes=False),zaxis=dict(showspikes=False, range=[z_range_min, z_range_max]),aspectmode="data")
    # Always use normal 3D perspective view (2D top-down view is shown separately)
    fig.update_layout(scene=dict(**scene, camera=dict(eye=dict(x=1.35, y=1.35, z=0.95), projection=dict(type="orthographic"))), margin=dict(l=0, r=0, t=100, b=100),legend=dict(orientation="h", y=-0.12, yanchor="top", x=0.5, xanchor="center"),title=f"3D Gradient Ascent — Step: {current_step}",width=780,height=780,uirevision="slider-3d")
    
    with out3d_slider:
        clear_output(wait=True)
        display(fig)
    
    # 2D path figure
    fig2 = go.Figure()
    f = surface_funcs[surface_dropdown_slider.value]
    Z_local = f(X, Y)
    fig2.add_trace(go.Heatmap(z=Z_local, x=x, y=y, colorscale="Viridis", showscale=False))
    fig2.add_trace(go.Contour(z=Z_local, x=x, y=y, showscale=False,
                              contours=dict(coloring="none", showlines=True),
                              line=dict(color="#777777", width=1)))
    # Path tail
    if len(path_x_visible) >= 2:
        fig2.add_trace(go.Scatter(x=path_x_visible, y=path_y_visible, mode="lines",
                                  line=dict(color="#e31a1c", width=3), name="path"))
    # Current point
    if len(path_x_visible) >= 1:
        fig2.add_trace(go.Scatter(x=[path_x_visible[-1]], y=[path_y_visible[-1]], mode="markers",
                                  marker=dict(size=8, color="#111111"), name="x(t)"))
    fig2.update_layout(xaxis_title="x", yaxis_title="y", title=f"Ascent path (2D) — Step: {current_step}",
                       width=260, height=260, margin=dict(l=40, r=10, t=40, b=40), showlegend=False)
    
    with out2d_slider:
        clear_output(wait=True)
        display(fig2)
    
    # Update status
    if len(path_x_visible) >= 1 and current_step < len(path_x_slider):
        status_html_slider.value = f"Step {current_step} / {len(path_x_slider)-1} (x={path_x_visible[-1]:.2f}, y={path_y_visible[-1]:.2f}, z={path_z_visible[-1]:.2f})"
    elif len(path_x_slider) > 0:
        status_html_slider.value = f"Step {current_step} / {len(path_x_slider)-1} (path not computed yet)"
    else:
        status_html_slider.value = "No path computed"

def _on_slider_change(change):
    """Handle slider change"""
    _render_with_path_slider()

def _on_start_point_change(change):
    """Handle starting point change - recompute path"""
    _compute_full_path_slider()
    step_slider.value = 0
    _render_with_path_slider()

def _on_surface_change(change):
    """Handle surface change - recompute path"""
    _compute_full_path_slider()
    step_slider.value = 0
    _render_with_path_slider()

# Wire up observers
step_slider.observe(_on_slider_change, names="value")
x0_input_slider.observe(_on_start_point_change, names="value")
y0_input_slider.observe(_on_start_point_change, names="value")
surface_dropdown_slider.observe(_on_surface_change, names="value")
show_bottom_heatmap_chk_slider.observe(_on_slider_change, names="value")
show_bottom_arrows_chk_slider.observe(_on_slider_change, names="value")
show_bottom_redlevel_chk_slider.observe(_on_slider_change, names="value")


# Build UI
bottom_table_title_slider = widgets.HTML("<b>Alter Bottom Plane</b>")
bottom_table_slider = widgets.VBox([
    widgets.HBox([show_bottom_heatmap_chk_slider, widgets.HTML("Level sets heatmap")], layout=widgets.Layout(align_items="center")),
    widgets.HBox([show_bottom_arrows_chk_slider, widgets.HTML("Gradient vector field")], layout=widgets.Layout(align_items="center")),
    widgets.HBox([show_bottom_redlevel_chk_slider, widgets.HTML("Red level set projection")], layout=widgets.Layout(align_items="center")),
], layout=widgets.Layout(align_items="flex-start"))

controls_row1_slider = widgets.HBox([surface_dropdown_slider])
plane_controls_mx_slider = widgets.VBox([
    bottom_table_title_slider,
    bottom_table_slider,
])
point_row_slider = widgets.HBox([widgets.HTML("<b>Starting Point:</b>&nbsp;"),x0_input_slider,y0_input_slider])
slider_row = widgets.HBox([step_slider, status_html_slider])

ui_slider = widgets.VBox([
    controls_row1_slider,
    plane_controls_mx_slider,

    point_row_slider,
    slider_row,
    widgets.HBox([out3d_slider, out2d_slider]),
])

# Initialize
_compute_full_path_slider()
_render_with_path_slider()
display(ui_slider)